In [1]:
#i "nuget:/Users/peet/Sources/csml/CsML/bin/Release"
// #i "nuget:C:\Users\peet.vermeulen\source\repos\csml\CsML\bin\Release"
#r "nuget:CsML,*-*"
using CsML;

Restore sources /Users/peet/Sources/csml/CsML/bin/Release Installed Packages CsML, 1.0.0

In [3]:
var mapping = new Dictionary<int, Dictionary<string, double>>();
mapping[60] = new Dictionary<string, double>{ { "R", 0 }, {"M", 1 }};
var inputPath = "/Users/peet/Sources/csml/Tests/Data/sonar.csv";
// inputPath = @"C:\Users\peet.vermeulen\source\repos\csml\Tests\Data\sonar.csv";
var data = CsML.Utility.Matrix.FromCSV(inputPath, mapping, loadFromRow: 0);

In [4]:
var dataLength = data.GetLength(0);
var features = new double[dataLength, 59];
var target = new double[dataLength];
for (int r=0; r < data.GetLength(0); r++)
{
    for (int c=0; c < 59; c++)
    {
        features[r, c] = data[r, c];
    }
    target[r] = data[r, 60];
}

In [5]:
var profiler = new CsML.Utility.Features.Profiler(features, target);
profiler.HasOutliers(features)

True

In [6]:
CsML.Utility.Features.ClassProportions<double>(target)

index,Item1,Item2,Item3
0,0,97,0.46634615384615385
1,1,111,0.5336538461538461


In [ ]:
(features, target) = CsML.Utility.Features.Shuffle(features, target);

In [7]:
var results = new List<double>(){};
var iter = new CsML.Utility.KFoldIterator(dataLength, 10);
double[,] ftrain, ftest;
double[] ttrain, ttest;
foreach(bool[] f in iter)
{    

    Console.Write(".");
    (ftrain, ftest) = CsML.Utility.Matrix.Split(features, f);
    (ttrain, ttest) = CsML.Utility.Array.Split(target, f);
    var rcfier = new CsML.Probability.Classification.RandomClassifier<double>();
    rcfier.Train(ftrain, ttrain);
    double[] predictions = rcfier.Predict(ftest);
    results.Add(CsML.Utility.Array.ClassificationAccuracy(ttest, predictions));
}
var mean = results.Average();
results = results.Select(x => Math.Round(x, 4)).ToList();
Console.WriteLine("");
Console.WriteLine(string.Join(", ", results.ToArray()));
Console.WriteLine($"Average {mean}");

..........
0.4, 0.5, 0.6, 0.75, 0.45, 0.5, 0.4, 0.45, 0.6, 0.35
Average 0.4999999999999999


In [8]:
results = new List<double>(){};
iter = new CsML.Utility.KFoldIterator(dataLength, 10);
foreach(bool[] f in iter)
{
    Console.Write(".");
    (ftrain, ftest) = CsML.Utility.Matrix.Split(features, f);
    (ttrain, ttest) = CsML.Utility.Array.Split(target, f);
    var tree = new CsML.Tree.BinaryTree("classify", CsML.Utility.Statistics.Gini);
    tree.maxdepth = 15;
    tree.minrows = 3;
    tree.Train(ftrain, ttrain);
    double[] predictions = tree.Predict(ftest);
    results.Add(CsML.Utility.Array.ClassificationAccuracy(ttest, predictions));
}
var mean = results.Average();
results = results.Select(x => Math.Round(x, 4)).ToList();
Console.WriteLine("");
Console.WriteLine(string.Join(", ", results.ToArray()));
Console.WriteLine($"Average {mean}");

..........
0.95, 0.55, 0.75, 0.55, 0.75, 0.6, 0.65, 0.55, 0.75, 0.8
Average 0.69


In [9]:
results = new List<double>(){};
iter = new CsML.Utility.KFoldIterator(dataLength, 10);
foreach(bool[] f in iter)
{
    Console.Write(".");
    (ftrain, ftest) = CsML.Utility.Matrix.Split(features, f);
    (ttrain, ttest) = CsML.Utility.Array.Split(target, f);
    var forest = new CsML.Tree.RandomForest("classify", CsML.Utility.Statistics.Gini);
    forest.treeCount = 500;
    forest.Train(ftrain, ttrain);
    double[] predictions = forest.Predict(ftest);
    results.Add(CsML.Utility.Array.ClassificationAccuracy(ttest, predictions));
}
var mean = results.Average();
results = results.Select(x => Math.Round(x, 4)).ToList();
Console.WriteLine("");
Console.WriteLine(string.Join(", ", results.ToArray()));
Console.WriteLine($"Average {mean}");

..........
0.95, 0.85, 0.9, 0.8, 0.75, 0.75, 0.8, 0.7, 0.75, 0.95
Average 0.82
